# Socar Fraud 데이터 검출 


1. 데이터 1차 전처리 
2. 모델 학습/예측/평가 
3. 성능 개선 - 전처리 
4. 성능 개선 - 이상치 제거
5. 성능 개선 - 오버 샘플링 


In [2]:
# 1. 데이터 1차 전처리 
# 1-1. 데이터 가져오기 
# 1-2. 데이터 스키밍 - 피처 확인, 결측치 제거 

In [7]:
# 1-1. 데이터 가져오기 

import pandas as pd 
import matplotlib.pyplot as plt 


socar = pd.read_csv('../../0. raw_data/insurance_fraud_detect_data.csv')
socar 

,fraud_YN,car_model,sharing_type,age_group,has_previous_accident,cumulative_use_count,b2b,accident_ratio,pf_type,socarpass,...,insure_cost,accident_location,car_part1,car_part2,repair_cnt,acc_type1,insurance_site_aid_YN,police_site_aid_YN,total_prsn_cnt,test_set
0,0,2,1,3,0,2,0,100,2,0,...,0,1,0,0,1,0,0,0,-1,0
1,0,1,0,4,0,4,0,0,3,1,...,0,1,1,1,2,1,1,1,0,1
2,0,1,0,4,1,4,0,100,3,1,...,12925040,3,1,0,2,1,1,1,0,1
3,0,3,1,3,0,4,0,0,2,0,...,0,3,1,0,1,0,0,0,-1,0
4,0,1,0,1,0,1,0,100,1,0,...,1209410,1,0,0,2,0,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0,2,0,2,0,2,0,100,2,0,...,0,1,1,0,1,0,0,0,-1,0
15996,0,2,0,2,1,4,0,100,1,1,...,0,2,1,0,1,0,0,0,-1,0
15997,0,2,1,2,0,4,0,100,1,0,...,0,0,0,0,1,0,0,0,-1,1
15998,0,2,0,2,0,2,0,100,2,0,...,0,4,1,0,1,0,0,0,-1,0


In [8]:
# 1-2. 데이터 스키밍 - 피처 확인, 결측치 제거 
socar.shape

(16000, 25)

In [9]:
socar.dtypes

fraud_YN                   int64
car_model                  int64
sharing_type               int64
age_group                  int64
has_previous_accident      int64
cumulative_use_count       int64
b2b                        int64
accident_ratio             int64
pf_type                    int64
socarpass                  int64
socarsave                  int64
start_hour                 int64
duration                   int64
accident_hour              int64
repair_cost              float64
insure_cost                int64
accident_location          int64
car_part1                  int64
car_part2                  int64
repair_cnt                 int64
acc_type1                  int64
insurance_site_aid_YN      int64
police_site_aid_YN         int64
total_prsn_cnt             int64
test_set                   int64
dtype: object

In [10]:
socar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fraud_YN               16000 non-null  int64  
 1   car_model              16000 non-null  int64  
 2   sharing_type           16000 non-null  int64  
 3   age_group              16000 non-null  int64  
 4   has_previous_accident  16000 non-null  int64  
 5   cumulative_use_count   16000 non-null  int64  
 6   b2b                    16000 non-null  int64  
 7   accident_ratio         16000 non-null  int64  
 8   pf_type                16000 non-null  int64  
 9   socarpass              16000 non-null  int64  
 10  socarsave              16000 non-null  int64  
 11  start_hour             16000 non-null  int64  
 12  duration               16000 non-null  int64  
 13  accident_hour          16000 non-null  int64  
 14  repair_cost            16000 non-null  float64
 15  in

#### 데이터 확인서 결측 데이터 없음. 피처 확인 시 연관 데이터로 전처리 진행하지 아니함 

In [24]:
# 2. 모델 학습/예측/평가 

# 데이터 분리 

from sklearn.model_selection import train_test_split

y_target = socar['fraud_YN']
X_features = socar.drop('fraud_YN', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target) #stratify 를 적용해서 train, test 분포를 비슷하게 처리 

# 분리된 레이블 값 비율 확인 
print(y_train.value_counts()/y_train.shape[0]*100) 
print(y_test.value_counts()/y_test.shape[0]*100)

0    99.742188
1     0.257812
Name: fraud_YN, dtype: float64
0    99.75
1     0.25
Name: fraud_YN, dtype: float64


In [28]:
# 모델링 (lr)

from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
lr_pred_proba = lr_clf.predict_proba(X_test)[:,1]

In [29]:
# 모델 평가 

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred) #오차행렬
    accuracy = accuracy_score(y_test,pred) # 정확도 
    precision = precision_score(y_test,pred) # 정밀도
    recall = recall_score(y_test,pred) # 재현율 
    f1 = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test,pred_proba)

    print('오차행렬')
    print(confusion)

    print('정확도 : {}, 정밀도 : {}, 재현률 : {}, f1 : {}, AUC : {}'.format(accuracy, precision, recall, f1, roc_auc))

In [30]:
# 함수 적용 

get_clf_eval(y_test, lr_pred, lr_pred_proba)

오차행렬
[[3192    0]
 [   8    0]]
정확도 : 0.9975, 정밀도 : 0.0, 재현률 : 0.0, f1 : 0.0, AUC : 0.46828007518797


In [ ]:
############## 재현률 0, 오버샘플링이나 다양한 전처리 진행할 것 ####################